<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h1 style="direction: rtl; unicode-bidi: plaintext; text-align: right; margin-top: 0;">
الوحدة M2.2 – العمل مع بيانات مناخية شبكية (Gridded)
</h1>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<strong>جزء من:</strong>
<a dir="ltr" href="https://github.com/OpenClimateScience/M2-Computational-Climate-Science" style="unicode-bidi: isolate;">
<strong>Computational Climate Science</strong>
</a>
&nbsp;|&nbsp;
<strong>الدرس السابق</strong>
&nbsp;|&nbsp;
<strong>الدرس التالي</strong>
</p>
</div>


<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<strong>المحتويات:</strong>
<ul style="list-style-position: inside; direction: rtl; unicode-bidi: plaintext; text-align: right;">
<li><a dir="ltr" href="#Conceptual-and-computational-constraints" style="unicode-bidi: isolate;">القيود المفاهيمية والحوسبية</a></li>
<li><a dir="ltr" href="#A-terrestrial-water-storage-(TWS)-time-series" style="unicode-bidi: isolate;">سلسلة زمنية لتخزين المياه الأرضية (TWS)</a>
  <ul style="list-style-position: inside;">
    <li><a dir="ltr" href="#Accessing-xarray-variables" style="unicode-bidi: isolate;">الوصول إلى متغيرات xarray</a></li>
  </ul>
</li>
<li><a dir="ltr" href="#Thinking-about-multi-dimensional-arrays" style="unicode-bidi: isolate;">التفكير في المصفوفات متعددة الأبعاد</a>
  <ul style="list-style-position: inside;">
    <li><a dir="ltr" href="#Updating-coordinate-systems" style="unicode-bidi: isolate;">تحديث أنظمة الإحداثيات</a></li>
  </ul>
</li>
<li><a dir="ltr" href="#Indexing-a-multi-dimensional-array" style="unicode-bidi: isolate;">فهرسة مصفوفة متعددة الأبعاد</a></li>
<li><a dir="ltr" href="#Aggregating-along-the-axis-of-a-multi-dimensional-array" style="unicode-bidi: isolate;">التجميع على طول محور مصفوفة متعددة الأبعاد</a></li>
<li><a dir="ltr" href="#Resampling-a-time-series" style="unicode-bidi: isolate;">إعادة أخذ العينات (Resampling) لسلسلة زمنية</a>
  <ul style="list-style-position: inside;">
    <li><a dir="ltr" href="#Handling-missing-data" style="unicode-bidi: isolate;">التعامل مع البيانات المفقودة</a></li>
  </ul>
</li>
<li><a dir="ltr" href="#Visualizing-variation-over-space" style="unicode-bidi: isolate;">تصوير التباين عبر المكان</a></li>
<li><a dir="ltr" href="#Summary" style="unicode-bidi: isolate;">ملخص</a></li>
</ul>
</div>


<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">القيود المفاهيمية والحوسبية</h2>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
مع تحسّن تقنيات الحوسبة وتزايد توفر مجموعات البيانات المعتمدة على الأقمار الصناعية، فإن حجم بيانات المناخ وتعقيدها وتواترها يتزايد بسرعة. وهذا قد يخلق مشكلتين للباحثين ومديري الموارد الراغبين في استخدام بيانات المناخ. أولًا، قد يكون من الصعب تصوّر الحجم والنطاق لبعض مجموعات البيانات المناخية، مما يؤدي إلى تحديات في إدارة البيانات والمشاريع (Jain وآخرون، 2022). ثانيًا، قد يكون من الصعب معالجة وتحليل مجموعات بيانات مناخية تكون أحيانًا كبيرة جدًا بحيث لا تتسع في ذاكرة الحاسوب، أو معقدة جدًا بحيث يصعب إجراء حسابات عملية عليها.
</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
في هذا الدرس، سنستكشف كيفية التعامل مع بعض هذه التحديات. وبما أننا في مرحلة التعلم، سنستخدم مجموعة بيانات ليست كبيرة أو معقدة مثل غيرها مما قد نواجهه، لكنها ستُعد مثالًا جيدًا لشرح مجموعات البيانات الأصعب وكيفية التعامل معها.
</p>
</div>


In [ ]:
import xarray as xr
import earthaccess
import numpy as np
from matplotlib import pyplot

auth = earthaccess.login()

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">سلسلة زمنية لتخزين المياه الأرضية (TWS)</h2>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
لاكتساب خبرة إضافية في التعامل مع البيانات المناخية الشبكية، وبشكل خاص السلاسل الزمنية للبيانات المناخية الشبكية، سنستخدم مجموعة بيانات لتخزين المياه الأرضية (TWS) تم إنتاجها باستخدام بيانات من بعثتين فضائيتين: بعثة تجربة استعادة الجاذبية والمناخ (GRACE) وخليفتها بعثة GRACE Follow-On (GRACE-FO).
</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
يشمل TWS جميع المياه على سطح اليابسة وتحتها، بما في ذلك الغطاء الثلجي، والجداول والخزانات السطحية، والمياه الجوفية.
وقد تم تحويل مجموعة بيانات TWS التي سنعمل عليها من بعثتي GRACE وGRACE-FO مسبقًا إلى <em>انحرافات شهرية</em>:
<span dir="ltr" style="unicode-bidi: isolate;">
<a dir="ltr" href="https://podaac.jpl.nasa.gov/dataset/TELLUS_GRAC-GRFO_MASCON_CRI_GRID_RL06.1_V3">https://podaac.jpl.nasa.gov/dataset/TELLUS_GRAC-GRFO_MASCON_CRI_GRID_RL06.1_V3</a>
</span>
</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
&#x1F449; اقرأ المزيد عن بيانات GRACE/GRACE-FO هنا:
<span dir="ltr" style="unicode-bidi: isolate;">
<a dir="ltr" href="https://grace.jpl.nasa.gov/about/faq/">https://grace.jpl.nasa.gov/about/faq/</a>
</span>
</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
تذكّر أن <em>الانحرافات (anomalies)</em> هي إحدى طرق تمثيل التذبذب حول متوسط طويل الأمد. فهي تخبرنا كيف يقارن قياسٌ معين (يوم أو شهر أو سنة) ببقية السلسلة الزمنية؛ مثلًا: هل هذه سنة جافة على نحوٍ غير معتاد؟
</p>
</div>


<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h4 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">&#x1F3AF; أفضل ممارسة</h4>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<em>نحن على وشك تنزيل بعض البيانات الخام! أنت تعرف ما ينبغي فعله.</em>
أنشئ مجلدًا جديدًا باسم <code dir="ltr" style="unicode-bidi: isolate;">GRACE</code> داخل مجلد <code dir="ltr" style="unicode-bidi: isolate;">data_raw</code>.
</p>
</div>


In [ ]:
results = earthaccess.search_data(short_name = 'TELLUS_GRAC-GRFO_MASCON_CRI_GRID_RL06.1_V3')
results

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
تم إعداد مجموعة البيانات هذه كملف netCDF واحد.
</p>
</div>


In [ ]:
earthaccess.download(results, 'data_raw/GRACE')

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<strong>قد تظهر رسالة خطأ عند محاولة تنزيل هذه المجموعة بسبب مشكلة في خوادم ناسا.</strong>
إذا حدث ذلك، فلاحظ أن لهذه المجموعة رابطًا واحدًا (مذكورًا في المخرجات أعلاه) يمكنك استخدامه لتنزيل البيانات مباشرة. ينبغي أن يكون:
</p>
<ul style="list-style-position: inside; direction: rtl; unicode-bidi: plaintext; text-align: right;">
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<span>اضغط للتنزيل:</span>
<span dir="ltr" style="unicode-bidi: isolate;">
<a dir="ltr" href="https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/TELLUS_GRAC-GRFO_MASCON_CRI_GRID_RL06.1_V3/GRCTellus.JPL.200204_202311.GLO.RL06.1M.MSCNv03CRI.nc">https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/TELLUS_GRAC-GRFO_MASCON_CRI_GRID_RL06.1_V3/GRCTellus.JPL.200204_202311.GLO.RL06.1M.MSCNv03CRI.nc</a>
</span>
</li>
</ul>
</div>


In [ ]:
ds = xr.open_dataset('data_raw/GRACE/GRCTellus.JPL.200204_202311.GLO.RL06.1M.MSCNv03CRI.nc')
ds

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
بيانات GRACE/GRACE-FO تمثل بالفعل <em>انحرافات</em>، أي تغيّرًا في إجمالي تخزين المياه مقارنةً بفترة زمنية مرجعية (أو "حقبة"). ما هي تلك الفترة؟ يمكننا معرفة ذلك من <strong>السمات (Attributes) على مستوى الملف</strong> في ملف netCDF هذا.
</p>
</div>


In [ ]:
ds.attrs['time_mean_removed']

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
يمكننا أيضًا التحقق من ذلك بالنظر إلى سلسلة زمنية للمتوسط العالمي للانحرافات الشهرية في TWS. خلال الفترة 2004–2009 تبدو الانحرافات متمركزة حول الصفر، وهو ما نتوقعه لأن متوسط تلك الفترة قد طُرح من البيانات.
</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
نلاحظ أيضًا اتجاهًا هبوطيًا طويل الأمد. ما الذي قد يفسر ذلك؟ من المرجّح أن يكون فقدان الجليد على اليابسة عالميًا هو العامل الرئيس وراء هذا الانخفاض. ويقدّر تقرير التقييم السادس (AR6) للهيئة الحكومية الدولية المعنية بتغير المناخ (IPCC) أن ذوبان الأنهار الجليدية والصفائح الجليدية خلال الفترة 2006–2018 أسهم بما يقارب سنتيمترين من ارتفاع مستوى سطح البحر (IPCC 2021، الفصل 9)، وهو ما يتسق مع ما نراه هنا (والمحور الرأسي في الرسم أيضًا بوحدة السنتيمتر). كما يلعب احترار الغلاف الجوي دورًا، إذ يمكن للهواء الدافئ حمل المزيد من بخار الماء.
</p>
</div>


In [ ]:
ds['lwe_thickness'].mean(['lon','lat']).plot()

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
الانقطاع الظاهر في السلسلة الزمنية حول 2017–2018 يعود إلى أن مهمة GRACE انتهت في أكتوبر 2017 ولم تُطلق GRACE-FO حتى مايو 2018؛ وبالتالي لا توجد قيم بيانات لتلك الأشهر.
</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
يمكننا النظر لأكثر من سلسلة زمنية: فهذه مجموعة بيانات مكانية أيضًا. دعنا نلقي نظرة على الانحرافات العالمية لتخزين المياه الأرضية (TWS) في أول تاريخ.
</p>
</div>


In [ ]:
ds['lwe_thickness'][0].plot(vmin = -200, vmax = 200, cmap = 'RdBu')

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
في الرسم أعلاه، الذي يعرض الانحراف الشهري لتخزين المياه الأرضية (TWS) لشهر أبريل 2002، تشير الألوان الباردة إلى انحراف موجب في TWS (أي مياه أكثر من المعتاد في أبريل)، وتشير الألوان الدافئة إلى انحراف سالب في TWS (أي مياه أقل من المعتاد في أبريل).
</p>
</div>


<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">الوصول إلى متغيرات <code dir="ltr" style="unicode-bidi: isolate;">xarray</code></h3>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
لاحظ أن كلًا من الإحداثيات (Coordinates) ومتغيرات البيانات (Data Variables) في كائنات <code dir="ltr" style="unicode-bidi: isolate;">xarray.Dataset</code> يمكن الوصول إليها بطريقتين مختلفتين:
</p>
<ul style="list-style-position: inside; direction: rtl; unicode-bidi: plaintext; text-align: right;">
<li>اسم الإحداثي أو المتغير كمفتاح في قاموس: <code dir="ltr" style="unicode-bidi: isolate;">ds['lwe_thickness']</code> أو <code dir="ltr" style="unicode-bidi: isolate;">ds['time']</code></li>
<li>اسم الإحداثي أو المتغير كخاصية (Attribute): <code dir="ltr" style="unicode-bidi: isolate;">ds.lwe_thickness</code> أو <code dir="ltr" style="unicode-bidi: isolate;">ds.time</code></li>
</ul>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
بغضّ النظر عن الصياغة التي تستخدمها، فالنتيجة واحدة.
</p>
</div>


<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<hr/>
<h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">التفكير في المصفوفات متعددة الأبعاد</h2>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<strong>لنُعد فتح مجموعة البيانات باستخدام <code dir="ltr" style="unicode-bidi: isolate;">open_mfdataset()</code>.</strong>
تذكّر أننا نستخدم <code dir="ltr" style="unicode-bidi: isolate;">open_mfdataset()</code> من <code dir="ltr" style="unicode-bidi: isolate;">xarray</code> عندما نريد فتح عدة ملفات وكأنها ملف واحد. هنا لدينا ملف واحد فقط، لكننا سنتعامل معه كما لو كان عدة ملفات لأنه يحتوي على صورة عالمية لانحرافات TWS لعدة تواريخ.
</p>
</div>


In [ ]:
ds = xr.open_mfdataset('data_raw/GRACE/GRCTellus.JPL.200204_202311.GLO.RL06.1M.MSCNv03CRI.nc')

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
كما رأينا، يمكن تمثيل ملفات netCDF4 وHDF5 ككائنات <code dir="ltr" style="unicode-bidi: isolate;">xarray.Dataset</code>.
</p>
</div>


In [ ]:
type(ds)

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
يمكن أن يحتوي Dataset على أكثر من متغير واحد (Variable)، ويُمثَّل كل متغير كمصفوفة متعددة الأبعاد.
</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
هناك طرق مختلفة لتمثيل المصفوفات في بايثون. وعند استخدام <code dir="ltr" style="unicode-bidi: isolate;">xarray</code>، يتم تمثيل المتغيرات كـ DataArrays.
</p>
</div>


In [ ]:
type(ds['lwe_thickness'])

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
عادةً ما يكون <code dir="ltr" style="unicode-bidi: isolate;">xarray.DataArray</code> مجرد نوع خاص من مصفوفات NumPy. لكن عند استخدام <code dir="ltr" style="unicode-bidi: isolate;">xarray.open_mfdataset()</code> نحصل على نوع جديد من المصفوفات.
</p>
</div>


In [ ]:
type(ds['lwe_thickness'].data)

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<strong>تُستخدم مكتبة <code dir="ltr" style="unicode-bidi: isolate;">dask</code> تلقائيًا لتمثيل المصفوفة متعددة الأبعاد الأساسية.</strong>
وذلك لأن استخدام <code dir="ltr" style="unicode-bidi: isolate;">open_mfdataset()</code> يوحي بأننا قد نتعامل مع مصفوفة كبيرة متعددة الأبعاد، لأننا نفتح عدة ملفات.
</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<strong>ومن خلال استخدام <code dir="ltr" style="unicode-bidi: isolate;">xarray.open_mfdataset()</code> لتكديس ملفات متعددة في سلسلة زمنية، اكتسبنا أيضًا طريقة جديدة للتفكير في بياناتنا.</strong>
فكل ملف فردي كان يمثل عمليًا صورة ثنائية الأبعاد: انحراف TWS على شبكة خطوط العرض والطول. ومع تكديس تواريخ متعددة للصور معًا، نحصل على <strong>مكعب بيانات</strong> ثلاثي الأبعاد بمحاور X (خط الطول) وY (خط العرض) والزمن. ويمكن أن تعرض كائنات <code dir="ltr" style="unicode-bidi: isolate;">xarray.Dataset</code> رسمًا توضيحيًا مفيدًا عند استخدامها داخل Jupyter Notebook.
</p>
</div>


In [ ]:
ds['lwe_thickness'].data

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
عند تحليل مكعبات البيانات، نحاول عادةً الإجابة عن أحد الأسئلة التالية:
</p>
<ol style="list-style-position: inside; direction: rtl; unicode-bidi: plaintext; text-align: right;">
<li>كيف يتغير متغير مناخي في موقع واحد أو عدة مواقع عبر الزمن؟</li>
<li>كيف يتغير متغير مناخي في وقت محدد عبر المكان؟</li>
<li>كيف تُقارن قابلية التذبذب المناخي (التغير عبر الزمن) بين المواقع المختلفة؟</li>
</ol>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
لا يتطلب السؤالان الأولان مكعب بيانات للإجابة عنهما، لكننا غالبًا ما نحتاج لدمج مجموعات بيانات مختلفة للإجابة. وفي هذه الحالة نكون محظوظين إذا بدأنا بمكعب بيانات لأن كل البيانات تكون في مكان واحد. تتطلب هذه الأسئلة اقتطاع البيانات أو <em>فهرسة</em> مصفوفة متعددة الأبعاد.
</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
أما السؤال الثالث فلا يمكن الإجابة عنه إلا إذا بدأنا بمكعب بيانات؛ لأنه يتطلب تجميع (أو <em>طيّ/إسقاط</em>) أحد محاور مكعب البيانات.
</p>
</div>


<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">تحديث أنظمة الإحداثيات</h3>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
تذكّر أن كائنات <code dir="ltr" style="unicode-bidi: isolate;">xarray.Dataset</code> تمتلك أنظمة إحداثيات ترمز للمعلومات المكانية للبيانات.
</p>
</div>


In [ ]:
ds.coords

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
يمكننا أن نرى أن بياناتنا تقع على شبكة خطوط عرض/طول بدقة نصف درجة؛ أي إن كل بكسل هو 0.5° × 0.5°.
</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
قد تكون لاحظت أن قيم خط العرض (<code dir="ltr" style="unicode-bidi: isolate;">lat</code>) تمتد من -90° إلى 90°، بينما قيم خط الطول (<code dir="ltr" style="unicode-bidi: isolate;">lon</code>) تمتد من 0° إلى 360°. نحن معتادون على تمثيل خطوط الطول من -180° (غربًا) إلى 180° (شرقًا). كيف يمكننا إصلاح ذلك؟
</p>
<h4 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">&#x1F6A9; <span style="color:red">انتبه</span></h4>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<strong>لنغيّر قيم <code dir="ltr" style="unicode-bidi: isolate;">lon</code> لتصبح أسهل في العمل.</strong>
سنطرح 180 درجة من الإحداثيات بحيث تصبح أصغر قيمة لخط الطول -180 وأكبر قيمة (التي كانت سابقًا تقارب 360) تصبح الآن 180.
</p>
</div>


In [ ]:
# Run this only once!
ds.coords['lon'] = ds.coords['lon'] - 180

In [ ]:
ds.coords

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<strong>الإحداثيات تشبه الملصقات (Labels) لنقاط البيانات في مكعب البيانات، بينما الأبعاد (Dimensions) هي محاور مكعب البيانات.</strong>
يمكننا تطبيق عدة ملصقات على البعد نفسه. على سبيل المثال، لدينا بعد باسم <code dir="ltr" style="unicode-bidi: isolate;">time</code> ولدينا أيضًا مجموعة إحداثيات باسم <code dir="ltr" style="unicode-bidi: isolate;">time</code>. إحداثي <code dir="ltr" style="unicode-bidi: isolate;">time</code> يصف اللحظة الزمنية بدقة حتى النانوثانية (ns)، لكن يمكننا أيضًا وسم نقاط الزمن حسب السنة أو الشهر أو الفصل أو اليوم.
</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<strong>لنُضف إحداثيًا جديدًا باسم <code dir="ltr" style="unicode-bidi: isolate;">year</code> إلى مجموعة البيانات كي نعرّف نقاط البيانات حسب السنة بشكل شائع.</strong>
سنستخرج أولًا السنة الرقمية من إحداثي <code dir="ltr" style="unicode-bidi: isolate;">"time"</code>. ثم نستخدم دالة <code dir="ltr" style="unicode-bidi: isolate;">assign_coords()</code> لإنشاء ملصقات إحداثيات جديدة على طول بعد موجود.
اقرأ المزيد هنا:
<span dir="ltr" style="unicode-bidi: isolate;">
<a dir="ltr" href="https://docs.xarray.dev/en/stable/generated/xarray.Dataset.assign_coords.html">https://docs.xarray.dev/en/stable/generated/xarray.Dataset.assign_coords.html</a>
</span>
</p>
</div>


In [ ]:
# Get a list of all the dates from the "time" coordinate
dates = ds.coords['time'].values

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
ألقِ نظرة على محتوى المتغير <code dir="ltr" style="unicode-bidi: isolate;">dates</code>:
</p>
<pre dir="ltr" style="unicode-bidi: isolate;"><code>dates</code></pre>
</div>


In [ ]:
# Convert each "datetime64[ns]" object to a 4-digit year
years_list = []
for each in dates:
    years_list.append(int(str(each)[0:4]))

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
ألقِ نظرة على محتوى المتغير <code dir="ltr" style="unicode-bidi: isolate;">years_list</code>:
</p>
<pre dir="ltr" style="unicode-bidi: isolate;"><code>years_list</code></pre>
</div>


In [ ]:
# Create a new coordinate ("year") along an existing dimension ("time")
ds = ds.assign_coords(year = ('time', years_list))
ds

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<hr/>
<h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">فهرسة مصفوفة متعددة الأبعاد</h2>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
يمكننا اقتطاع مكعب البيانات بعدة طرق للإجابة عن أسئلة مختلفة. على سبيل المثال: كيف تبدو انحرافات TWS خلال العشرين سنة الماضية في غرب الولايات المتحدة، بالنظر إلى الجفاف متعدد العقود الذي شهدته المنطقة؟
</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
لنبدأ بالنظر إلى البكسل الذي يحتوي مدينة ساكرامنتو، كاليفورنيا. إن اقتطاع مصفوفتنا عند هذا البكسل الواحد يعادل أخذ شريط رفيع على طول محور الزمن؛ وكما هو موضح، نحصل عمليًا على مصفوفة أحادية البعد من 220 قيمة (تمثل 220 شهرًا من البيانات).
</p>
</div>


In [ ]:
ds['lwe_thickness'].sel(lon = -121.75, lat = 38.25).data

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
هذا يعادل اقتطاع المصفوفة على طول محوري X وY (خط الطول وخط العرض)، كما نرى أدناه.
</p>
</div>


In [ ]:
# Three dimensions: (time, lat, lon)
ds['lwe_thickness'].shape

In [ ]:
# Take the first pixel along the "lat" dimension and the first pixel along the "lon" dimension
ds['lwe_thickness'][:,0,0].data

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
يمكن تمثيل البيانات كسلسلة زمنية لأن لدينا قيمة واحدة عبر الزمن.
</p>
</div>


In [ ]:
ds['lwe_thickness'].sel(lon = -121.75, lat = 38.25).plot(figsize = (12, 6))

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<strong>إذا أردنا النظر إلى عدة بكسلات تمثل منطقة اهتمام، فنحن ما زلنا نقتطع المصفوفة على طول محور الزمن، لكننا ننتهي بمكعب بيانات أصغر.</strong>
</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
تذكّر أن الدالة المدمجة <code dir="ltr" style="unicode-bidi: isolate;">slice()</code> يمكن استخدامها مع <code dir="ltr" style="unicode-bidi: isolate;">sel()</code> في <code dir="ltr" style="unicode-bidi: isolate;">xarray</code> لاختيار منطقة اهتمام. في هذه الحالة، منطقتنا بعرض 10 بكسلات وارتفاع 10 بكسلات.
</p>
</div>


In [ ]:
west_us = ds['lwe_thickness'].sel(lon = slice(-124, -114), lat = slice(32, 42))
west_us.data

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
لا توجد طريقة سهلة لعرض قيم مكعب بيانات ثلاثي الأبعاد، لذلك عندما نطلب من <code dir="ltr" style="unicode-bidi: isolate;">xarray</code> رسم البيانات، فإنه يعرض لنا مخططًا تكراريًا (Histogram)، أي أنه يجمع كل قيم مكعب البيانات معًا.
</p>
</div>


In [ ]:
west_us.plot()

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
وبالطبع، يمكننا دائمًا رسم خطوة زمنية واحدة عبر اقتطاع مجموعة البيانات.
</p>
</div>


In [ ]:
west_us.sel(time = '2003-01-01', method = 'nearest').plot()

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
من شكل هذه الصورة (وإذا فحصنا قيم المصفوفة الأساسية)، يمكننا الاستنتاج أنه رغم أن دقة شبكة خطوط العرض/الطول هي 0.5 درجة، فإن القيم تتكرر في بعض المناطق لأن
الدقة المكانية الفعلية للبيانات أقرب إلى 300 كم.
اقرأ المزيد هنا:
<span dir="ltr" style="unicode-bidi: isolate;">
<a dir="ltr" href="https://grace.jpl.nasa.gov/about/faq/">https://grace.jpl.nasa.gov/about/faq/</a>
</span>
</p>
</div>


<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<hr/>
<h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">التجميع على طول محور مصفوفة متعددة الأبعاد</h2>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
كما يشير المخطط التكراري أعلاه، نحتاج إلى طريقة لتحويل مكعب البيانات كي نعرض البيانات بشكل أفضل ونجيب عن بعض أسئلتنا.
</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<strong>ما متوسط انحراف TWS في منطقة الدراسة لدينا في كل شهر؟</strong>
يمكن الإجابة عن ذلك بأخذ المتوسط عبر المجال المكاني. ويمكن تصور ذلك على أنه أخذ متوسط لكل شريحة (10×10 بكسل) من مكعب البيانات، لكل خطوة زمنية شهرية.
</p>
</div>


In [ ]:
west_us.data

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
إذا حدّدنا أننا نريد أخذ المتوسط عبر أحد محاورنا المكانية، فإننا ننتقل من مصفوفة ثلاثية الأبعاد إلى مصفوفة ثنائية الأبعاد؛ أي <strong>قمنا بطيّ/إسقاط أحد المحاور</strong>.
</p>
</div>


In [ ]:
west_us.mean('lat').data

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
تحديدًا، فقدنا البعد <code dir="ltr" style="unicode-bidi: isolate;">"lat"</code> كما نرى أدناه.
</p>
</div>


In [ ]:
west_us.mean('lat').dims

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
لنأخذ المتوسط عبر محورين مكانيين: عبر خط العرض وخط الطول معًا. هذا يعني أننا ننتقل من مصفوفة ثلاثية الأبعاد إلى مصفوفة أحادية البعد؛ ورغم أنها تُعرض أدناه كمصفوفة ثنائية الأبعاد، إلا أن لدينا محورًا تافهًا طوله 1، لذا فهي فعليًا سلسلة أحادية البعد من 220 قيمة.
</p>
</div>


In [ ]:
west_us.mean(['lat', 'lon']).data

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
وبما أننا حصلنا على سلسلة زمنية أحادية البعد لانحرافات TWS، فيمكننا عرضها كمخطط خطي. ومن هذا الرسم، يمكننا أن نرى أن الجفاف متعدد العقود في غرب الولايات المتحدة قد ازداد سوءًا في السنوات الأخيرة.
</p>
</div>


In [ ]:
west_us.mean(['lat', 'lon']).plot(figsize = (12, 6))

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<hr/>
<h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">إعادة أخذ العينات (Resampling) لسلسلة زمنية</h2>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
مهمة تحليل بيانات شائعة أخرى في السلاسل الزمنية هي إعادة تجميع البيانات؛ على سبيل المثال: هل يمكننا حساب متوسط انحراف TWS <em>السنوي</em> لوصف السنوات الرطبة والجافة بشكل أفضل؟
</p>
</div>


In [ ]:
west_us.data

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
تُعد دالة <code dir="ltr" style="unicode-bidi: isolate;">resample()</code> في <code dir="ltr" style="unicode-bidi: isolate;">xarray.DataArray</code> أفضل طريقة لتجميع بيانات السلاسل الزمنية.
اقرأ المزيد هنا:
<span dir="ltr" style="unicode-bidi: isolate;">
<a dir="ltr" href="https://docs.xarray.dev/en/stable/generated/xarray.DataArray.resample.html">https://docs.xarray.dev/en/stable/generated/xarray.DataArray.resample.html</a>
</span>
</p>
</div>


In [ ]:
west_us_annual = west_us.resample(time = 'YS').mean()
west_us_annual.data

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
الصياغة <code dir="ltr" style="unicode-bidi: isolate;">time = 'YS'</code> أعلاه تشير إلى استخدام <em>بداية السنة</em> (YS اختصارًا لـ Year Start) كتواتر لإعادة أخذ العينات؛ أي حساب المتوسط لكل سنة. هذه الصياغة تأتي من مكتبة <code dir="ltr" style="unicode-bidi: isolate;">pandas</code>.
</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
&#x1F449; <strong>اقرأ المزيد عن</strong> <code dir="ltr" style="unicode-bidi: isolate;">resample()</code> وتواتر إعادة أخذ العينات هنا:
<span dir="ltr" style="unicode-bidi: isolate;">
<a dir="ltr" href="https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects">https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects</a>
</span>
</p>
</div>


<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
ينبغي أيضًا استخدام <code dir="ltr" style="unicode-bidi: isolate;">resample()</code> في بعض الحسابات مثل حساب الاتجاهات الزمنية، حيث تكون وحدات الاتجاه مقسومة على وحدة الزمن (مثل "لكل يوم" أو "لكل سنة"). وذلك لأن <code dir="ltr" style="unicode-bidi: isolate;">coarsen()</code> لا يغيّر وحدة الزمن؛ أي أنه لا يحوّلها مثلًا من أيام إلى سنوات.
</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
عمومًا، مع محاور السلاسل الزمنية، من الأفضل استخدام <code dir="ltr" style="unicode-bidi: isolate;">resample()</code> على DataArray.
</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
لكن لاحظ أنه عند إعادة أخذ العينات على طول بعد معيّن، لا يتم الاحتفاظ بكل ملصقات الإحداثيات. في هذه الحالة، لا يعرف <code dir="ltr" style="unicode-bidi: isolate;">xarray</code> كيف يعيّن ملصق <code dir="ltr" style="unicode-bidi: isolate;">year</code> للإحداثيات المُعاد أخذ عيناتها.
</p>
</div>


In [ ]:
# We lost the "year" coordinate
west_us_annual.coords

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h4 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">&#x1F6A9; <span style="color:red">انتبه</span></h4>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
لكن في هذه الحالة تحديدًا، بما أننا مهتمون بالتجميع حسب السنة أصلًا، فلا يلزم استخدام <code dir="ltr" style="unicode-bidi: isolate;">resample()</code>؛ يمكننا استخدام <code dir="ltr" style="unicode-bidi: isolate;">groupby()</code> للإشارة إلى أننا نريد أخذ المتوسط داخل كل ملصق لإحداثي <code dir="ltr" style="unicode-bidi: isolate;">"year"</code>.
</p>
</div>


In [ ]:
west_us_annual = west_us.groupby('year').mean()
west_us_annual

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">التعامل مع البيانات المفقودة</h3>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
مع مجموعات البيانات المعتمدة على الأقمار الصناعية مثل مجموعة TWS هذه، هناك دائمًا احتمال لوجود بيانات مفقودة. على سبيل المثال، توجد فجوة في 2017 و2018 عند انتهاء مهمة GRACE قبل إطلاق القمر GRACE-FO. وهناك أيضًا أشهر مفقودة في 2002 قبل إطلاق GRACE في مارس، كما أن بيانات GRACE-FO لعام 2023 لا تتضمن ديسمبر 2023 لأنها ما تزال قيد المعالجة.
</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
&#x1F449; <strong>ملاحظة:</strong> لأنك تصل إلى هذه البيانات من سحابة ناسا، فقد تجد أن بيانات أحدث (مع أشهر مفقودة أقل) متاحة!
</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
أدناه، سنعدّ عدد الأشهر المتاحة في كل سنة.
</p>
</div>


In [ ]:
counts = ds.groupby('year').count()

# Get a value (the count) in each year for an arbitrary pixel, skipping the first year (2002)
counts['lwe_thickness']

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<strong>لكننا نحصل على عدد الأشهر في كل بكسل!</strong>
نحن نريد فقط العدد لبكسل واحد (اختياري) لأن توفر البيانات في هذا المنتج المُنمذج (TWS) هو نفسه لكل البكسلات.
</p>
</div>


In [ ]:
# Get the count at an arbitrary pixel: the top-left (0,0) corner
counts_by_year = counts['lwe_thickness'][:,0,0].values
counts_by_year

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
من الواضح أننا نفتقد عدة أشهر من البيانات، خصوصًا في 2017 و2018 (حيث تتوفر 5 أشهر فقط في كل سنة) بين بعثتي GRACE وGRACE-FO.
</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
ينبغي أولًا التأكد من توفر بعض البيانات على الأقل في كل سنة. أدناه، نرى أن محور الزمن لدينا يحتوي على 21 عنصرًا (21 سنة). وهذا يتوافق مع أعداد الأشهر عندما نتجاوز السنة الأولى (2002).
</p>
</div>


In [ ]:
west_us_annual.shape

In [ ]:
counts_by_year.size

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<strong>لنضع قاعدة: نحتاج إلى 9 أشهر على الأقل من البيانات في كل سنة للحصول على متوسط سنوي موثوق.</strong>
</p>
</div>


In [ ]:
mask = counts_by_year < 9
mask

In [ ]:
west_us_annual[mask] = np.nan

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
من المفيد تذكّر أن إصلاحًا مرتبطًا بهذه المشكلة — إذا كانت كل نقاط البيانات الصحيحة متتالية — هو أخذ شريحة (Slice):
</p>
</div>


In [ ]:
# A related fix, if the valid data points were all consecutive
west_us_annual = west_us_annual.sel(year = slice(2003, 2023))
west_us_annual.year

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;"><hr/></div>

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
إن تجميع الفواصل الزمنية هو مجرد واحد من الأشياء الرائعة العديدة التي يمكن لـ <code dir="ltr" style="unicode-bidi: isolate;">xarray.DataArray</code> القيام بها!
اطّلع على توثيق <code dir="ltr" style="unicode-bidi: isolate;">xarray</code> لمعرفة المزيد:
<span dir="ltr" style="unicode-bidi: isolate;">
<a dir="ltr" href="https://docs.xarray.dev/en/stable/generated/xarray.DataArray.html">https://docs.xarray.dev/en/stable/generated/xarray.DataArray.html</a>
</span>
</p>
</div>


In [ ]:
west_us_annual.mean(['lat','lon']).plot()

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
يمكننا ملاحظة اتجاه هبوطي عام في البيانات، وهو ما يتسق مع الجفاف طويل الأمد الذي شهدته المنطقة على مدى عقود (Liu وآخرون، 2022).
</p>
</div>


<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<hr/>
<h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">تصوير التباين عبر المكان</h2>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
في المثال السابق، أعدنا تجميع بياناتنا عبر المحور الزمني (أي عبر الزمن). لكن لمجموعة البيانات لدينا إحداثيات مكانية أيضًا! غالبًا ما نريد تصوير كيف يختلف التباين الزمني بين المواقع المختلفة. وهذا ببساطة يعني إعادة أخذ عينات/تجميع مكعب البيانات بطريقة مختلفة.
</p>
</div>


In [ ]:
west_us_annual.data

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
يمكننا حساب متوسط بسيط بين السنوات. وهذا يطوي مكعب البيانات إلى بُعدين مكانيين: خط العرض وخط الطول.
</p>
</div>


In [ ]:
west_us_annual.mean('year').data

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<strong>لكن سيكون الأمر أكثر إثارة للاهتمام إذا استطعنا حساب اتجاهات على مستوى البكسل في الانحرافات السنوية لـ TWS.</strong>
</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
وبما أننا نعمل مع <code dir="ltr" style="unicode-bidi: isolate;">xarray</code> — وهي مكتبة بايثون فعالة وناضجة للعمل مع المصفوفات متعددة الأبعاد — فمن الجيد التحقق إن كانت الوظيفة التي نريدها مدمجة بالفعل في <code dir="ltr" style="unicode-bidi: isolate;">xarray.DataArray</code>. بالفعل، هناك دالتان قد تساعدان هنا:
</p>
<ul style="list-style-position: inside; direction: rtl; unicode-bidi: plaintext; text-align: right;">
<li><code dir="ltr" style="unicode-bidi: isolate;">DataArray.curvefit()</code> — التوثيق: <span dir="ltr" style="unicode-bidi: isolate;"><a dir="ltr" href="https://docs.xarray.dev/en/stable/generated/xarray.DataArray.curvefit.html#xarray.DataArray.curvefit">https://docs.xarray.dev/en/stable/generated/xarray.DataArray.curvefit.html#xarray.DataArray.curvefit</a></span></li>
<li><code dir="ltr" style="unicode-bidi: isolate;">DataArray.polyfit()</code> — التوثيق: <span dir="ltr" style="unicode-bidi: isolate;"><a dir="ltr" href="https://docs.xarray.dev/en/stable/generated/xarray.DataArray.polyfit.html#xarray.DataArray.polyfit">https://docs.xarray.dev/en/stable/generated/xarray.DataArray.polyfit.html#xarray.DataArray.polyfit</a></span></li>
</ul>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
يمكن استخدام أي منهما لملاءمة دالة لمجموعة من نقاط البيانات. لكننا سنستخدم اليوم <code dir="ltr" style="unicode-bidi: isolate;">polyfit()</code> لأننا نلائم دالة بسيطة (اتجاه خطي) يفترض أن لها حلًا مغلق الصيغة. كثير الحدود من الدرجة 1 هو خط مستقيم لأن الدالة تكون على شكل: <span dir="ltr" style="unicode-bidi: isolate;">y(x) = mx + b</span>، حيث يُرفع <span dir="ltr" style="unicode-bidi: isolate;">x</span> للأس 1.
</p>
</div>


In [ ]:
fit = west_us_annual.polyfit('year', deg = 1)
fit

In [ ]:
fit['polyfit_coefficients']

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<strong>لقد حصلنا على مكعب بيانات آخر!</strong>
هذا المكعب يحتوي على محور باسم <strong>degree</strong> فيه عنصران (2). رغم أننا طلبنا ملاءمة كثير حدود بدرجة 1، تذكّر أننا نحتاج رقمين لوصف خط مستقيم: نقطة التقاطع مع محور y (y-intercept) والميل (slope).
</p>
</div>


<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h4 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">&#x1F6A9; <span style="color:red">انتبه</span></h4>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
ملاحظة أخرى حول مكعب البيانات لدينا: لدينا خط اتجاه (نقطة تقاطع وميل) لكل واحد من 400 بكسل. بدت تلك الانحدارات الخطية الـ 400 سريعة جدًا!
</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<strong>في الواقع، لم نقم بحساب الانحدارات الخطية بعد.</strong>
هذا لأننا حمّلنا بيانات TWS باستخدام <code dir="ltr" style="unicode-bidi: isolate;">xr.open_mfdataset()</code>. هذه الدالة تُشير تلقائيًا إلى <code dir="ltr" style="unicode-bidi: isolate;">xarray</code> أننا قد نتعامل مع مجموعة بيانات كبيرة جدًا؛ لذلك لا يجري <code dir="ltr" style="unicode-bidi: isolate;">xarray</code> أي حساب فعلي حتى نطلب ذلك صراحة.
</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
ماذا حدث عندما كتبنا:
</p>
<pre dir="ltr" style="unicode-bidi: isolate;"><code>fit = west_us_annual.polyfit('year', deg = 1)</code></pre>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
ما حدث أننا حصلنا على <strong>تمثيل</strong> للحساب الذي نريد من <code dir="ltr" style="unicode-bidi: isolate;">xarray</code> تنفيذه. سمّينا هذا التمثيل <code dir="ltr" style="unicode-bidi: isolate;">fit</code> ويبدو كمكعب بيانات لأنه سيكون شكل الناتج عند تنفيذ الحساب فعليًا.
</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<strong>وعندما نكون جاهزين لتنفيذ الحساب فعليًا، نحتاج إلى استدعاء <code dir="ltr" style="unicode-bidi: isolate;">compute()</code>.</strong>
اقرأ المزيد هنا:
<span dir="ltr" style="unicode-bidi: isolate;"><a dir="ltr" href="https://docs.xarray.dev/en/stable/generated/xarray.DataArray.compute.html">https://docs.xarray.dev/en/stable/generated/xarray.DataArray.compute.html</a></span>
</p>
</div>


In [ ]:
# Get a representation of the computation we want to do
fit = west_us_annual.polyfit('year', deg = 1)

# Actually run the computation
results = fit.compute()

# Look at the coefficients
results['polyfit_coefficients']

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
هذا لأن <code dir="ltr" style="unicode-bidi: isolate;">xarray</code> يستخدم <strong>التقييم الكسول (Lazy Evaluation)</strong>؛ إذ لا تُحمّل البيانات إلى الذاكرة ولا تُعالج إلا عند الحاجة. يساعد ذلك على تجنب عمليات القراءة والكتابة غير الضرورية والتي تكون غالبًا بطيئة.
</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<strong>هناك طريقتان لضمان تنفيذ الحسابات على Datasets وDataArrays في <code dir="ltr" style="unicode-bidi: isolate;">xarray</code> عندما تريد:</strong>
</p>
<ul style="list-style-position: inside; direction: rtl; unicode-bidi: plaintext; text-align: right;">
<li>استدعاء <code dir="ltr" style="unicode-bidi: isolate;">compute()</code> كما في المثال أعلاه.</li>
<li>أو محاولة الوصول إلى الخاصية <code dir="ltr" style="unicode-bidi: isolate;">values</code> كما في المثال أدناه.</li>
</ul>
<pre dir="ltr" style="unicode-bidi: isolate;"><code>fit['polyfit_coefficients'].values</code></pre>
</div>


<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<hr/>
<h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">ملخص</h2>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
فيما يلي أهم أدوات وتقنيات بايثون التي ينبغي تذكرها من هذا الدرس:
</p>
<ul style="list-style-position: inside; direction: rtl; unicode-bidi: plaintext; text-align: right;">
<li>
يمكنك استخدام <code dir="ltr" style="unicode-bidi: isolate;">xarray.open_mfdataset()</code> لفتح عدة ملفات كـ Dataset واحد. مثلًا قد تكتب:
<pre dir="ltr" style="unicode-bidi: isolate;"><code>data = xarray.open_mfdataset("precip_in_year_Y*.nc")</code></pre>
لفتح سلسلة من ملفات بيانات الهطول السنوية.
</li>
<li>يمكن الوصول إلى السمات (Attributes) أو البيانات الوصفية (Metadata) لأي ملف فُتح في <code dir="ltr" style="unicode-bidi: isolate;">xarray</code> عبر: <code dir="ltr" style="unicode-bidi: isolate;">data.attrs</code></li>
<li>
تُسمّى Datasets في <code dir="ltr" style="unicode-bidi: isolate;">xarray</code> أحيانًا <strong>مكعبات بيانات</strong>. ويمكنك التجميع عبر محور أو عدة محاور (أبعاد) باستخدام دوال مثل <code dir="ltr" style="unicode-bidi: isolate;">mean()</code>؛ مثلًا لأخذ المتوسط عبر البعدين المكانيين لخط الطول وخط العرض:
<pre dir="ltr" style="unicode-bidi: isolate;"><code>data['variable'].mean(['lon','lat'])</code></pre>
</li>
<li>
لاقتطاع مكعب البيانات على طول محور أو عدة محاور، استخدم <code dir="ltr" style="unicode-bidi: isolate;">sel()</code>، مثلًا:
<pre dir="ltr" style="unicode-bidi: isolate;"><code>data['variable'].sel(lon = -121.75, lat = 38.25)</code></pre>
</li>
<li>
تُستخدم <code dir="ltr" style="unicode-bidi: isolate;">resample()</code> لتجميع بيانات السلاسل الزمنية، مثل تجميع بيانات كل ساعة إلى يومية أو بيانات شهرية إلى سنوية.
اقرأ المزيد عن تواتر إعادة أخذ العينات هنا:
<span dir="ltr" style="unicode-bidi: isolate;"><a dir="ltr" href="https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects">https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects</a></span>
</li>
<li>
يستخدم <code dir="ltr" style="unicode-bidi: isolate;">xarray</code> <strong>التقييم الكسول</strong> لتجنب القراءة/الكتابة/الحساب حتى اللحظة الأخيرة؛ ولإجبار <code dir="ltr" style="unicode-bidi: isolate;">xarray</code> على التحميل أو التقييم يمكنك استدعاء <code dir="ltr" style="unicode-bidi: isolate;">compute()</code> أو الوصول إلى <code dir="ltr" style="unicode-bidi: isolate;">values</code>.
</li>
</ul>
</div>


<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<hr/>
<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">موارد إضافية</h3>
<ul style="list-style-position: inside; direction: rtl; unicode-bidi: plaintext; text-align: right;">
<li>
الحوسبة على Datasets وDataArrays في <code dir="ltr" style="unicode-bidi: isolate;">xarray</code>:
<span dir="ltr" style="unicode-bidi: isolate;"><a dir="ltr" href="https://docs.xarray.dev/en/stable/user-guide/computation.html">https://docs.xarray.dev/en/stable/user-guide/computation.html</a></span>
</li>
</ul>
<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">المراجع</h3>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
IPCC. <em>Climate Change 2021: The Physical Science Basis. Contribution of Working Group I to the Sixth Assessment Report of the Intergovernmental Panel on Climate Change.</em> 2021. Cambridge University Press, Cambridge, United Kingdom and New York, NY, USA, 2391 pp. doi:10.1017/9781009157896
</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
Jain S, Mindlin J, Koren G, Gulizia C, Steadman C, Langendijk GS, Osman M, Abid MA, Rao Y, Rabanal V. 2022.
<span>المقال:</span>
<span dir="ltr" style="unicode-bidi: isolate;"><a dir="ltr" href="https://doi.org/10.1029/2022AV000676">https://doi.org/10.1029/2022AV000676</a></span>
<span> (AGU Advances)</span>
</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
Liu, P.-W., J. S. Famiglietti, A. J. Purdy, K. H. Adams, A. L. McEvoy, J. T. Reager, R. Bindlish, D. N. Wiese, C. H. David, and M. Rodell. 2022.
<span>المقال:</span>
<span dir="ltr" style="unicode-bidi: isolate;"><a dir="ltr" href="https://www.nature.com/articles/s41467-022-35582-x">https://www.nature.com/articles/s41467-022-35582-x</a></span>
<span> (Nature Communications 13(1):7825)</span>
</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<code dir="ltr" style="unicode-bidi: isolate;">xarray.DataArray</code> (توثيق xarray):
<span dir="ltr" style="unicode-bidi: isolate;"><a dir="ltr" href="https://docs.xarray.dev/en/stable/generated/xarray.DataArray.html">https://docs.xarray.dev/en/stable/generated/xarray.DataArray.html</a></span>
<span> — تاريخ الوصول: 5 فبراير 2024.</span>
</p>
</div>
